In [2]:
#from nytimesarticle import articleAPI
#api = articleAPI('a0HA3uBISDkGyvUGR3FeoAGybtDVPPM5')

In [4]:
#articles = api.search( q = 'immigration', 
#     begin_date = 20120101, end_date =20120201 )

# create TIDF on an example call from R

https://towardsdatascience.com/latent-semantic-analysis-sentiment-classification-with-python-5f657346f6a3

In [26]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
import nltk
import gensim 
from gensim.models import Word2Vec 
from gensim.models.wrappers import FastText 

from gensim.models import FastText

In [11]:
df = pd.read_csv("allNYTSearch20122019hits.txt", sep = "\t", encoding = 'unicode_escape')

In [12]:
df

,status,copyright,response.docs.web_url,response.docs.snippet,response.docs.lead_paragraph,response.docs.abstract,response.docs.print_page,response.docs.source,response.docs.pub_date,response.docs.document_type,...,response.docs.headline.content_kicker,response.docs.headline.print_headline,response.docs.headline.name,response.docs.headline.seo,response.docs.headline.sub,response.docs.byline.original,response.docs.byline.organization,response.meta.hits,response.meta.offset,response.meta.time
1,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/2019/09/11/us/politics...,The Democrats seeking the White House despise ...,WASHINGTON  One Democratic candidate would po...,The Democrats seeking the White House despise ...,24,The New York Times,2019-09-11T20:40:32+0000,article,...,NaN,"On Immigration Policy, Democrats Are Hazy On t...",NaN,NaN,NaN,By Zolan Kanno-Youngs,NaN,1459,0,44.0
2,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/reuters/2019/10/22/wor...,Greek authorities moved more refugees and migr...,ATHENS  Greek authorities moved more refugees...,Greek authorities moved more refugees and migr...,NaN,Reuters,2019-10-22T14:32:47+0000,article,...,NaN,"Greece Moves More Migrants to Mainland, Warns ...",NaN,NaN,NaN,By Reuters,Reuters,1459,0,44.0
3,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/aponline/2019/10/16/wo...,Greece's eastern islands are struggling to cop...,"SKALA, Greece  Greece's eastern islands are s...",Greece's eastern islands are struggling to cop...,NaN,AP,2019-10-16T08:18:12+0000,article,...,NaN,AP PHOTOS: Migrant Surge Overwhelms Greek Islands,NaN,NaN,NaN,By The Associated Press,The Associated Press,1459,0,44.0
4,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/2019/09/14/world/europ...,President Trumps plan to bar migrants from se...,BERLIN  The Supreme Court this week allowed t...,President Trumps plan to bar migrants from se...,18,The New York Times,2019-09-14T16:00:06+0000,article,...,NaN,"Trump Asylum Hurdle Is Tough, but Not Unique",NaN,NaN,NaN,By Patrick Kingsley,NaN,1459,0,44.0
5,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/2019/09/27/opinion/edi...,Slashing the number of refugees admitted to th...,President Trumps latest assault on immigratio...,Slashing the number of refugees admitted to th...,22,The New York Times,2019-09-28T01:00:26+0000,article,...,NaN,Mr. Trump Tears Up the Welcome Mat,NaN,NaN,NaN,By The Editorial Board,The Editorial Board,1459,0,44.0
6,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/2019/08/13/arts/design...,Without exiles and émigrés there is no modern ...,"WASHINGTON  In the first place, we dont lik...",Without exiles and émigrés there is no modern ...,1,The New York Times,2019-08-13T09:00:12+0000,article,...,NaN,The Museum Is the Refugees Home,NaN,NaN,NaN,By Jason Farago,NaN,1459,0,44.0
7,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/2019/06/28/us/politics...,He attributed the drop to Mexico cracking down...,"WASHINGTON  Kevin McAleenan, the acting secre...",He attributed the drop to Mexico cracking down...,NaN,The New York Times,2019-06-28T23:36:43+0000,article,...,NaN,NaN,NaN,NaN,NaN,By Zolan Kanno-Youngs,NaN,1459,0,44.0
8,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/reuters/2019/10/15/wor...,A fire in a severely overcrowded migrant camp ...,ATHENS  A fire in a severely overcrowded migr...,A fire in a severely overcrowded migrant camp ...,NaN,Reuters,2019-10-15T19:27:43+0000,article,...,NaN,Second Greek Migrant Camp in Flames as Arrival...,NaN,NaN,NaN,By Reuters,Reuters,1459,0,44.0
9,OK,Copyright (c) 2019 The New York Times Company....,https://www.nytimes.com/2019/09/08/world/europ...,The European Union is preparing to pay Rwanda ...,"BRUSSELS  For three years, the European Union...",The European Union is preparing to pay Rwanda

In [17]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    #stem_words=[stemmer.stem(w) for w in filtered_words]
    #lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)
    




In [18]:
df['cleanText']=df['response.docs.lead_paragraph'].map(lambda s:preprocess(s)) 

In [20]:
Tokenized_lead_paragraphs = df.apply(lambda row: nltk.word_tokenize(row['cleanText']), axis=1)

In [43]:
model_by_category = gensim.models.Word2Vec(Tokenized_lead_paragraphs, min_count = 1,  
                               window = 15, sg = 0) #CBOW method, the idea is that given a context, we want to know which word is most likely to appear in it

vocabulary = model_by_category.wv.vocab
#print(vocabulary)


sim_words = model_by_category.wv.most_similar('camps', topn = 50)
print(sim_words)

[('people', 0.9993313550949097), ('asylum', 0.9993246793746948), ('long', 0.9993223547935486), ('day', 0.999321460723877), ('country', 0.9993060231208801), ('migrants', 0.9993043541908264), ('immigrants', 0.9993024468421936), ('america', 0.9992980360984802), ('refugees', 0.9992973208427429), ('already', 0.9992964863777161), ('american', 0.9992961287498474), ('migration', 0.9992940425872803), ('home', 0.9992936849594116), ('administration', 0.9992930293083191), ('east', 0.99929279088974), ('toward', 0.9992885589599609), ('mexico', 0.9992868900299072), ('many', 0.9992859363555908), ('crisis', 0.9992855787277222), ('back', 0.9992841482162476), ('border', 0.9992824792861938), ('left', 0.9992784261703491), ('new', 0.9992742538452148), ('thousands', 0.9992724061012268), ('seekers', 0.9992704391479492), ('united', 0.9992702603340149), ('would', 0.9992687702178955), ('tuesday', 0.9992684125900269), ('political', 0.9992672801017761), ('city', 0.999266505241394), ('way', 0.9992644786834717), ('m

In [27]:
model_most_similar_words_lead_P = FastText(Tokenized_lead_paragraphs, size=100, window=5, min_count=5)

In [47]:
model_most_similar_words_lead_P.wv.most_similar("migration", topn = 50)

[('immigration', 0.9999982118606567),
 ('separation', 0.9999963045120239),
 ('nation', 0.9999953508377075),
 ('generation', 0.9999950528144836),
 ('population', 0.9999949932098389),
 ('station', 0.9999948740005493),
 ('destination', 0.9999947547912598),
 ('deportation', 0.9999946355819702),
 ('nomination', 0.9999940991401672),
 ('action', 0.9999937415122986),
 ('national', 0.9999934434890747),
 ('administration', 0.9999933242797852),
 ('investigation', 0.999992847442627),
 ('legislation', 0.9999925494194031),
 ('protection', 0.9999924898147583),
 ('detention', 0.9999924898147583),
 ('situation', 0.9999923706054688),
 ('international', 0.999992311000824),
 ('deportations', 0.9999922513961792),
 ('election', 0.9999922513961792),
 ('attention', 0.9999921321868896),
 ('coalition', 0.9999920725822449),
 ('education', 0.9999920129776001),
 ('organization', 0.9999919533729553),
 ('nations', 0.9999915957450867),
 ('nationalist', 0.9999914169311523),
 ('question', 0.9999913573265076),
 ('soluti